# 7. Loglinear models for contingency tables and counts

- Loglinear models are GLMs for count data 
- expresses count/rates in terms of categorial or continuous covariates

## 7.1 Loglinear models for counts in contingency tables

Setting 
- $(r \times c)$ contingency tables that categorizes $n$ subjects into response categories $X$ and $Y$ 
- if $X$ and $Y$ are indep, joint cell probabilities $\pi_{ij} = P(X=i, Y=j) = P(X=i)P(Y=j) = \pi_{i,:} \pi_{:,j}$
  - $\pi_{ij}$ are parameters for multinomial
- loglinear models use $\mu_{ij} = n\pi_{ij}$ and Poisson distribution
- log link function 
- do not distinguish b/w response and explanatory variables 

### 7.1.1 Loglinear model of independence for 2-way contingency tables

$$
\log \mu_{ij} = \lambda + \lambda_i^X + \lambda_j^Y
$$

where 
- $\lambda = \log n$
- $\lambda_i^X = \log \pi_{i,:}$ is the row effect of $X$ via row $i$
- $\lambda_j^Y = \log \pi_{:,j}$ is the column effect of $Y$ via column $j$
- the value of $\lambda_i \propto$ expected frequency in i-th row/column
- null hypothesis of independence uses expected frequencies from chi-squared tests of independence, 
  - ie $\mu_{ij} = n_{i,:}n_{:,j}/n$ 

Loglinear models do not distinguish between response and explanatory variables
- instead, attempts to model solely the cell counts rather than their individual classifications

### 7.1.2 Interpretation of parameters in the independence model
Take a $r \times 2$ contingency table. The logit in the $i$-th row is 
$$
\begin{align}
logit[P(Y=1)] &= \log \frac{P(Y=1)}{1 - P(Y=1)} \\
    &= \log \frac{\pi_{i,1}}{\pi_{i,2}} \\
    &= (\lambda + \lambda_i^X + \lambda_1^Y) - 
    (\lambda + \lambda_i^X + \lambda_2^Y) \\
    &= \lambda_1^Y - \lambda_2^Y
\end{align}
$$
- tHe logit for $Y$ is indep of the level of $X$ 
- for each row, the odds of response in the $i$-th column is the same 

In [2]:
HappyHeaven <-  read.table(
    "http://users.stat.ufl.edu/~aa/cat/data/HappyHeaven.dat",
    header = TRUE, stringsAsFactors = TRUE)
head(HappyHeaven)

,happy,heaven,count
,<fct>,<fct>,<int>
1,not,no,32
2,not,yes,190
3,pretty,no,113
4,pretty,yes,611
5,very,no,51
6,very,yes,326


Let $X$ be happiness rating (not, pretty, very) and $Y$ be whether respondents believed in heaven (yes, no). When fitting a loglinear model, we use a Poisson family, whose canonical link is the log link, so we don't write `family=poisson(link=log)`, as this is implied

In [3]:
fit <- glm(count ~ happy + heaven, family=poisson, data=HappyHeaven)
summary(fit)


Call:
glm(formula = count ~ happy + heaven, family = poisson, data = HappyHeaven)

Deviance Residuals: 
       1         2         3         4         5         6  
-0.15570   0.06459   0.54947  -0.23152  -0.65897   0.27006  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  3.49313    0.09408   37.13  < 2e-16 ***
happypretty  1.18211    0.07672   15.41  < 2e-16 ***
happyvery    0.52957    0.08460    6.26 3.86e-10 ***
heavenyes    1.74920    0.07739   22.60  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 1019.87238  on 5  degrees of freedom
Residual deviance:    0.89111  on 2  degrees of freedom
AIC: 49.504

Number of Fisher Scoring iterations: 3


The difference in deviances is ~1000, so the equivalent likelihood ratio test would suggest that the model fits well. Note that, for each response, there is one category whose log odds is zero, e.g. $\lambda_1^Y = 0$ and $\lambda_2^Y = 1.749$. The odds of belief in heaven was $\exp(1.749)$ times for each happiness category.

### 7.1.4 Saturated model for 2-way contingency tables

We can add associations between categorical variables
$$
\log \mu_{ij} = \lambda + \lambda_i^X + \lambda_j^Y + \lambda_{ij}^{XY}
$$
These associations are related to the log odds ratio 
$$
\begin{align}
\log \theta &= \log \frac{\mu_{11}\mu_{22}}{\mu_{12}\mu_{21}}\\
&= \log \mu_{11} + \mu_{22} - \mu_{12} - \mu_{21} \\
&= (\lambda + \lambda_1^X + \lambda_1^Y + \lambda_{11}^{XY}) \\
&\quad + (\lambda + \lambda_2^X + \lambda_2^Y + \lambda_{22}^{XY}) \\
&\quad - (\lambda + \lambda_1^X + \lambda_2^Y + \lambda_{12}^{XY}) \\
&\quad - (\lambda + \lambda_2^X + \lambda_1^Y + \lambda_{21}^{XY}) \\
&= \lambda_{11}^{XY} + \lambda_{22}^{XY} - \lambda_{12}^{XY} - \lambda_{21}^{XY}
\end{align}
$$

A $r \times c$ contingency table only has $(r-1)(c-1)$ non-redundant parameters (see [Chapter 2 notes](./2_AnalyzingContingencyTables_full.ipynb)). 
- `R` sets first row and column probabilities to zero, leaving $\log \theta = \lambda_{22}^{XY}$. 
  - ie chisq tests of indep have $(r-1)(c-1)$ dofs 
- in addn to the $(r-1)(c-1)$ probabilities above, the model params include
  - $\lambda = \log n$
  - $r-1$ nonredundant $\lambda_i^X$s and $c-1$ $\lambda_j^Y$s, ie one per category 
  - total = $1 + (r-1) + (c-1) + (r-1)(c-1) = rc$ 
    - ie same number of params as there are cells
    - aka **saturated loglinear model**
    - deviance = 0, bc models every set of cell counts 


In [23]:
fit2 <- glm(
    count ~ happy + heaven + happy:heaven, 
    family=poisson, data=HappyHeaven
)
summary(fit2)


Call:
glm(formula = count ~ happy + heaven + happy:heaven, family = poisson, 
    data = HappyHeaven)

Deviance Residuals: 
[1]  0  0  0  0  0  0

Coefficients:
                      Estimate Std. Error z value Pr(>|z|)    
(Intercept)            3.46574    0.17678  19.605  < 2e-16 ***
happypretty            1.26165    0.20025   6.300 2.97e-10 ***
happyvery              0.46609    0.22552   2.067   0.0388 *  
heavenyes              1.78129    0.19108   9.322  < 2e-16 ***
happypretty:heavenyes -0.09358    0.21679  -0.432   0.6660    
happyvery:heavenyes    0.07378    0.24329   0.303   0.7617    
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 1.0199e+03  on 5  degrees of freedom
Residual deviance: 2.9532e-14  on 0  degrees of freedom
AIC: 52.613

Number of Fisher Scoring iterations: 2


### 7.1.5 Loglinear models for 3-way contingency tables
- may have two-factor association terms, eg $\lambda_{ik}^{XZ}$ and $\lambda_{jk}^{YZ}$
  - eg. $XZ$ = interactions b/w X and Z for each category of Y
- can model conditional indep, e.g. have XY and ZY terms, but not XZ 
  - ie XZ association after we adjust/condn X and Z on Y 
- only single factor terms (X, Y, Z) aka *mutual independence model*
- model with all 3 2-factor terms: **homogeneous association** 
  - ie conditional odds ratios b/w any two variables are the same at each category of the third
- saturated model also includes 3-factor XYZ term 

### 7.1.6 2-factor parameters describe conditional associations
If we select a partial table $k$ of $Z$, ie for a given category of $Z$, the log odds ratio becomes
$$
\log \theta_{XY(k)} = \lambda_{22}^{XY}
$$
following a similar argument as above. 
- $\log \theta$ does not depend on $k$, so is identical for each partial table, ie every category of $Z$ 
- eg XZ odds ratios are the same for each category of Y, and YZ '...' for X

In [29]:
Drugs <- read.table("http://users.stat.ufl.edu/~aa/cat/data/Substance.dat",
                        header = TRUE, stringsAsFactors = TRUE)

head(Drugs)                       

,alcohol,cigarettes,marijuana,count
,<fct>,<fct>,<fct>,<int>
1,yes,yes,yes,911
2,yes,yes,no,538
3,yes,no,yes,44
4,yes,no,no,456
5,no,yes,yes,3
6,no,yes,no,43


2x2x2 contingency table with 
- A = alcohol use
- C = cigarette use
- M = marijuana use 

We fit the following models
- (A, C, M) = only single factor terms
- (AM, CM) = A and C are conditionally independent, given M
- (AC, AM, CM) = 3-way independent
- (ACM) = saturated

In [30]:
library(dplyr)

Drugs <- Drugs %>% 
  rename(A = "alcohol") %>% 
  rename(C = "cigarettes") %>% 
  rename(M = "marijuana")

head(Drugs)

,A,C,M,count
,<fct>,<fct>,<fct>,<int>
1,yes,yes,yes,911
2,yes,yes,no,538
3,yes,no,yes,44
4,yes,no,no,456
5,no,yes,yes,3
6,no,yes,no,43


In [32]:
A_C_M <- glm(
    count ~ A+C+M, family=poisson, data=Drugs
)
AM_CM <- glm(
    count ~ A+C+M + A:M + C:M, family=poisson, data=Drugs
)
AC_AM_CM <- glm(
    count ~ A+C+M + A:M + C:M + A:C, family=poisson, data=Drugs
)
ACM <- glm(
    count ~ A+C+M + A:M + C:M + A:C + A:C:M, family=poisson, data=Drugs
)

In [34]:

get_results <- function (FIT, NDIGITS=1, DATA=Drugs) {
    df <- round(exp(predict(
        FIT, data.frame(DATA)
    )), NDIGITS)
    return(df)
}

`(A_C_M)` <- get_results(A_C_M)
`(AM, CM)` <- get_results(AM_CM, NDIGITS=2)
`(AC_AM_CM)` <- get_results(AC_AM_CM)
`(ACM)` <- get_results(ACM)

`Table 7.2` <- bind_cols(
    `Alcohol` = c("Yes", rep("", 3), "No", rep("", 3)),
    `Cigarettes` = rep(c("Yes", "", "No", ""), 2), 
    `Marijuana` = rep(c("Yes", "No"), 4),
    `(A, C, M)` = `(A_C_M)`,
    `(AM, CM)` = `(AM, CM)`,
    `(AC, AM, CM)` = `(AC_AM_CM)`,
    `(ACM)` = `(ACM)`
)

library(knitr)
knitr::kable(`Table 7.2`)



|Alcohol |Cigarettes |Marijuana | (A, C, M)| (AM, CM)| (AC, AM, CM)| (ACM)|
|:-------|:----------|:---------|---------:|--------:|------------:|-----:|
|Yes     |Yes        |Yes       |     540.0|   909.24|        910.4|   911|
|        |           |No        |     740.2|   438.84|        538.6|   538|
|        |No         |Yes       |     282.1|    45.76|         44.6|    44|
|        |           |No        |     386.7|   555.16|        455.4|   456|
|No      |Yes        |Yes       |      90.6|     4.76|          3.6|     3|
|        |           |No        |     124.2|   142.16|         42.4|    43|
|        |No         |Yes       |      47.3|     0.24|          1.4|     2|
|        |           |No        |      64.9|   179.84|        279.6|   279|

A marginal odds ratio ignores the third factor, whereas conditional odds ratio adjust for it, e.g. the AC conditional odds ratio for model $(AM, CM)$ is the $AC$ odds ratio for each category of $M$ 


| Alcohol | Cigarettes | (AM, CM) |
| ------- | ---------- | -------- |
| Yes     | Yes        | 909.24        |
| .       | No         | 45.76        |
| No      | Yes        | 4.76        |
| .       | No         | 0.24        |

$$\theta^{(AM, CM)}_{AC \mid M} = \frac{909.24(0.24)}{45.76(4.76)}$$

However, the marginal table for $AC$ would sum over the values of $M$:

| Alcohol | Cigarettes | (AM, CM) |
| ------- | ---------- | -------- |
| Yes     | Yes        | 909.24+438.84        |
| .       | No         | 45.76+555.16        |
| No      | Yes        | 4.76+142.16        |
| .       | No         | 0.24+179.84        |

The marginal association of $AC$ would thus be different.

In [37]:
parse_fit_coefs <- function (FIT, COLS, NDIGITS=1) {
    return(round(exp(coef(FIT)[COLS]), 1))
}

In [38]:
line2 <- parse_fit_coefs(
    AM_CM, c("Ayes:Myes", "Cyes:Myes")
)
line3 <- parse_fit_coefs(
    AC_AM_CM, c("Ayes:Cyes", "Ayes:Myes", "Cyes:Myes"),
)
line4 <- parse_fit_coefs(
    ACM, c("Ayes:Cyes", "Ayes:Myes", "Cyes:Myes")
)

In [42]:
`Table 7.3` <- bind_cols(
    Model = c("(A, C, M)", "(AM, CM)", "(AC, AM, CM)", "(ACM)"), 
    matrix(
        c(
            rep(1.0, 3), 
            1.0, line2["Ayes:Myes"], line2["Cyes:Myes"],
            line3["Ayes:Cyes"], line3["Ayes:Myes"], line3["Cyes:Myes"],
            line4["Ayes:Cyes"], line4["Ayes:Myes"], line4["Cyes:Myes"]
        ), 
        ncol=3, byrow=T, dimnames=list(NULL, c("AC", "AM", "CM"))
    )
)

knitr::kable(`Table 7.3`)



|Model        |  AC|   AM|   CM|
|:------------|---:|----:|----:|
|(A, C, M)    | 1.0|  1.0|  1.0|
|(AM, CM)     | 1.0| 61.9| 25.1|
|(AC, AM, CM) | 7.8| 19.8| 17.3|
|(ACM)        | 7.7| 13.5|  9.7|

Note that the conditional odds ratio for $AC$ for model $(AM, CM)$ is 1.0. This makes sense, b/c this model assumes $A$ and $C$ are conditionally independent, given $M$, and thus does not have an $AC$ term. 
- ie conditional odds ratios = 1.0 for all interaction terms that do not appear in a model 

As shown above, this is different from the marginal associations, which require marginalizing/summing values for each category of $M$. 
- ie **conditional independence does not imply marginal independence**

In [47]:
summary(AC_AM_CM)


Call:
glm(formula = count ~ A + C + M + A:M + C:M + A:C, family = poisson, 
    data = Drugs)

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
 0.02044  -0.02658  -0.09256   0.02890  -0.33428   0.09452   0.49134  -0.03690  

Coefficients:
            Estimate Std. Error z value Pr(>|z|)    
(Intercept)  5.63342    0.05970  94.361  < 2e-16 ***
Ayes         0.48772    0.07577   6.437 1.22e-10 ***
Cyes        -1.88667    0.16270 -11.596  < 2e-16 ***
Myes        -5.30904    0.47520 -11.172  < 2e-16 ***
Ayes:Myes    2.98601    0.46468   6.426 1.31e-10 ***
Cyes:Myes    2.84789    0.16384  17.382  < 2e-16 ***
Ayes:Cyes    2.05453    0.17406  11.803  < 2e-16 ***
---
Signif. codes:  0 '***' 0.001 '**' 0.01 '*' 0.05 '.' 0.1 ' ' 1

(Dispersion parameter for poisson family taken to be 1)

    Null deviance: 2851.46098  on 7  degrees of freedom
Residual deviance:    0.37399  on 1  degrees of freedom
AIC: 63.417

Number of Fisher Scoring iterati

Note how the coefficient for `Ayes:Cyes` is 2.054, which is exactly the conditioanl odds ratio we found in Table 7.2 above. This is bc
$$
\log \theta_{XY} = \lambda_{22}^{XY}
$$
for any $XY$, for models such as $(AC, AM, CM)$ or simpler ones. 

## 7.2 Statistical inference for loglinear models
- goodness of fit stats like chisq only give global indications of lack of fit
- cell residuals show fit quality per cell, ie can use standardized residuals

In [48]:
deviance(AM_CM); deviance(AC_AM_CM)

[1] 187.7543

[1] 0.3739859

In [49]:
res <- rstandard(AM_CM, type="pearson")
res2 <- rstandard(AC_AM_CM, type="pearson")
data.frame(
    Drugs$A, Drugs$C, Drugs$M, Drugs$count, 
    fitted(AM_CM), res, 
    fitted(AC_AM_CM), res2
)

,Drugs.A,Drugs.C,Drugs.M,Drugs.count,fitted.AM_CM.,res,fitted.AC_AM_CM.,res2
,<fct>,<fct>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
1,yes,yes,yes,911,909.2395833,3.695518,910.38317,0.6333249
2,yes,yes,no,538,438.8404255,12.804588,538.61683,-0.6333249
3,yes,no,yes,44,45.7604167,-3.695518,44.61683,-0.6333249
4,yes,no,no,456,555.1595745,-12.804588,455.38317,0.6333249
5,no,yes,yes,3,4.7604167,-3.695589,3.61683,-0.6333250
6,no,yes,no,43,142.1595745,-12.804589,42.38317,0.6333249
7,no,no,yes,2,0.2395833,3.695589,1.38317,0.6333250
8,no,no,no,279,179.8404255,12.804589,279.61683,-0.6333249


The (AM, CM) model assumes A and C are conditionally indep
- 2 dofs, which explains why there are 2 nonredundant standardized residuals (3.7, 12.8)
- large positive residual when both A and C are yes or no
  - ie more students used both AC or neither AC than expected if usage of A or C were conditionally independent
- large negative residual when A and C don't match
  - ie less students used A, but not C, or vice versa, than expected if the usage of A and C were conditionally independent

We can test whether a specific term is needed by comparing the deviances of models with and without the term. 

In [63]:
library(car)
deviance(AM_CM); deviance(AC_AM_CM)
Anova(AC_AM_CM)

[1] 187.7543

[1] 0.3739859

,LR Chisq,Df,Pr(>Chisq)
,<dbl>,<dbl>,<dbl>
A,1281.71373,1,1.064401e-280
C,227.81432,1,1.786535e-51
M,55.91296,1,7.575145e-14
A:M,91.64437,1,1.037404e-21
C:M,496.99529,1,4.282907e-110
A:C,187.38032,1,1.186237e-42


Confidence intervals
- when the highest-order terms are 2-factor associations, estimated coefficients correspond to conditional log odds ratios

In [64]:
exp(confint(AC_AM_CM))

Waiting for profiling to be done...



,2.5 %,97.5 %
(Intercept),2.481623e+02,313.62395491
Ayes,1.404993e+00,1.89112888
Cyes,1.087964e-01,0.20616472
Myes,1.700562e-03,0.01136462
Ayes:Myes,8.814046e+00,56.64359514
Cyes:Myes,1.264576e+01,24.06925090
Ayes:Cyes,5.601452e+00,11.09714777


The table above shows 95% profile likelihood CIs. 
- eg `Ayes:Myes` $\in [8.8, 56.6]$ implies that, for each category of $C$ (cigarette usage), the estimated odds of alcohol is 8-56x greater for those who have smoke marijuana versus those who haven't, indicating strong positive conditional association b/w $A$ and $M$ 
- the AC odds also has strong positive association
- ie strong tendency for users of one to use another, whether or not they use the third 

In [65]:
Accidents <- read.table("http://users.stat.ufl.edu/~aa/cat/data/Accidents2.dat",
                        header = TRUE, stringsAsFactors = TRUE)
head(Accidents)                        

,gender,location,seatbelt,injury,count
,<fct>,<fct>,<fct>,<fct>,<int>
1,female,rural,no,no,3246
2,female,rural,no,yes,973
3,female,rural,yes,no,6134
4,female,rural,yes,yes,757
5,female,urban,no,no,7287
6,female,urban,no,yes,996


Accidents classified by passenger gender (G), location (L), seat belt use (S), and injury (I). Cell counts are proportions of passengers who were injured. 

In [66]:
Accidents <- Accidents %>%
    rename(G = "gender") %>%
    rename(L = "location") %>% 
    rename(S = "seatbelt") %>%
    rename(I = "injury") %>%
    rename(y = "count")

head(Accidents)

,G,L,S,I,y
,<fct>,<fct>,<fct>,<fct>,<int>
1,female,rural,no,no,3246
2,female,rural,no,yes,973
3,female,rural,yes,no,6134
4,female,rural,yes,yes,757
5,female,urban,no,no,7287
6,female,urban,no,yes,996


In [69]:
# G*I = G + I + G:I
fit <- glm(y ~ G*L*S + G*I + L*I + S*I, family = poisson, data = Accidents)
summary(fit)


Call:
glm(formula = y ~ G * L * S + G * I + L * I + S * I, family = poisson, 
    data = Accidents)

Deviance Residuals: 
       1         2         3         4         5         6         7         8  
-0.15190   0.27851   0.51823  -1.44646   0.16160  -0.43483  -0.42327   1.69037  
       9        10        11        12        13        14        15        16  
-0.34700   0.83292  -0.05675   0.20564   0.21675  -0.76754   0.09329  -0.49684  

Coefficients:
                  Estimate Std. Error z value Pr(>|z|)    
(Intercept)        8.08784    0.01654 488.884  < 2e-16 ***
Gmale              0.63640    0.02015  31.579  < 2e-16 ***
Lurban             0.80411    0.01966  40.891  < 2e-16 ***
Syes               0.62713    0.02027  30.940  < 2e-16 ***
Iyes              -1.21640    0.02649 -45.918  < 2e-16 ***
Gmale:Lurban      -0.28274    0.02441 -11.584  < 2e-16 ***
Gmale:Syes        -0.54186    0.02590 -20.925  < 2e-16 ***
Lurban:Syes       -0.15752    0.02441  -6.453 1.09e-10 ***
Gmale:I

The coefficient $SI$ has estimate $-0.817$
- $\exp(-0.817) = 0.44$, ie odds of injury for those wearing seatbelts were 0.44x those who did not, for every gender-location combination 
- we can repeat this interpretation for terms $LI$ and $GI$, but not for $GL$, GS, or LS 
- this is b/c we have the GLS term, which implies lack of 3-way independence
  - eg GS odds ratio varies with L 
- to compute odds ratios such as GL and GS, we need to do $GL(L=i)$ for every category $i \in L$. 
  - eg $GS(L=\text{urban}) = \beta_{GS} + \beta_{GLS} = -0.542 + 0.129$ 
  - eg $GS(L=\text{rural}) = \beta_{GS} + 0 = \beta_{GS} = -0.542$

Dissimilarity index
$$
D = \sum \frac{|n_i - \hat{\mu}_i|}{2n} = \sum \frac{|p_i - \hat{\pi}_i|}{2}
$$
- does not depend on sample size
  - useful bc measures like AIC, goodness of fit tests, etc. depend on sample size $n$
- $D \in [0,1]$

In [76]:
DI <- sum(abs(Accidents$y - fitted(fit)))/(2*sum(Accidents$y))
round(DI, 5)

[1] 0.00251

## 7.3 Loglinear-Logistic Model connection
- loglinear models = associations b/w categorical variables
- logistic models = set of explanatory variables on a categorical response
- conversion
  - can take logits of a response in a loglinear model
  - logistic models whose explanatory variables are all categorical have equivalent loglinear models
    - eg if logistic model only has single-factors, then the equivalent loglinear model will have all single-factors plus the fullest interaction term 

Take the following logistic model
$$
logit[P(I=1)] = \alpha + \beta^G + \beta^L + \beta^S 
$$
- G, L, and S are all associated with I, but not interacting 
- the equivalent loglinear is the saturated loglinear model containing all 1-, 2-, and the 3- factor terms 
- the log odds ratio for the effect of S on I is given by $\beta_1^S - \beta_2^S$ ,which is equivalent to $\lambda^{SI}_{11} + \lambda^{SI}_{22} - \lambda^{SI}_{12} - \lambda^{SI}_{21}$ 



Loglinear vs logistic
- loglinear = more than one categorical response
- if some marginal totals are fixed (eg by design), the model should include those margins as terms
  - eg if $n_{g+,l+}$ are fixed for each combination of $G$ and $L$, then a loglinear model should contain the $GL$ term so that $\mu_{g+,l+} = n_{g+,l+}\pi_{g+,l+} = n_{g+,l+}$

## 7.4 Independence graphs and collapsibility
Independence graph for a log linear model
- set of vertices which each represent a variable
  - no. vertices = no. dimensions of contingency table
- missing edge b/w vertices = the two variables are conditionally independent
- undirected 
- path = sequence of edges b/w variables
- two variables X and Y are separated by a subset of variables if all paths b/w X and Y intersect the subset 

> **Conditional independence and separation**
> two variables are conditionally independent given any subset of variables that separates them 

Collapsibility conditions
- conditional indep does not always imply marginal independence
- collapsibility conditions = when odds ratios for a model are identical in partial tables as in marginal table

> XY marginal and conditional odds ratios are identical if Z is conditionally independent from
> 1. either X or Y, or 
> 2. both X and Y


